In [3]:
import pandas as pd
df = pd.read_csv("small_df.csv",delimiter=',')
feature_df = pd.read_csv("tfidf_matrix.csv",delimiter=',')

In [4]:
feature_df.drop(columns=['Unnamed: 0'], inplace=True)

In [5]:
#load summary
import csv
with open("Summaries_of_restaurants.csv", "r", newline="") as csvfile:
  reader = csv.reader(csvfile)
  summary = {}
  summ = []
  for row in reader:
    summ.append(row[1])
  i=1
  for key in df:
    summary[key] = summ[i]
    i = i+1

In [6]:
def initilizing():
    corpus = []#all possible valid input words 
    for key in feature_df:
        corpus.append(key)

    return corpus

In [7]:
#rank all restaurants, get their scores, which store in 'ranks'
#input:a list of string(words from all filters)
#output:a dictionary, value is the score of restaurant
def ranking(filter_words, corpus):
    ranks = {}
    for key in df:
        ranks[key] = 0
    n = 0
    for word in filter_words: 
        if(word in corpus): 
            i = 0
            for key in ranks:
                ranks[key] = (n*ranks[key]+feature_df[word][i])/(n+1)
                i = i+1
        n = n+1
    return ranks

In [8]:
#used by submit and next button
#input:next_time is default when first time called , and a list of four filter words
#next_time is the time that next button is pressed
#output:a string(name of a restaurant)
def submitting_and_next(filter_words=[], next_time=0):
    corpus = initilizing()
    ranks = ranking(filter_words, corpus)
    #pick the (next_times+1)th best restaurant
    for i in range(next_time+1):
        recommend = "十六區和風料理"
        highest_score = ranks["十六區和風料理"]
        for key in ranks:
            if (ranks[key] > highest_score):
                recommend = key
                highest_score = ranks[key]
        ranks[recommend] = 0

    if(highest_score == 0):
        return {"Restaurant":"No other recommendation","Summary":"NULL"}
    return {"Restaurant":recommend,"Summary":summary[recommend]}

## DEMO

In [9]:
filter_words = ["flavor","beef"]

In [10]:
submitting_and_next(filter_words)

{'Restaurant': '如意麵食館 牛肉麵',
 'Summary': 'Pumpkin beef soup is delicious,noodle itself was just ok,not special. Peanut beef roll was also tasty,especially skin was the best in Hsinchu,with peanut butter inside. Not the best beef noodle Ive had, but definitely the most creative. They also sell different soup base beef noodles, like tomato and pumpkin.'}

In [11]:
# first time use next
submitting_and_next(filter_words,1)

{'Restaurant': 'Shi Tzi Beef Noodles',
 'Summary': 'The beef noodles were good and had a decent amount of tender meat for 120NT. The store is known for their rolls though, so make sure to try those! Although a little bit dry, they were full of flavor and very tasty. On a Friday around 6:30 there was around a ten minute wait.'}

In [20]:
# second time use next
for i in range(2,5):
    print(submitting_and_next(filter_words,i))


{'Restaurant': 'Duan Chun Zhen', 'Summary': 'I have the beef noodle here.The taste is really nice, serve with a big chunk of beef.The soup is a bit too salty but overall I still love the taste here. The price is pretty expensive compared to other shops. The beef noodles of Dian Chin Zhen must be enjoyed here in Hsinchu.'}
{'Restaurant': '八分飽餐廳', 'Summary': 'Good solid food. Clean tasting with subtle flavors for my Western palette. My favorite was the sour vegetables with beef wraps. Free all you can eat ice cream. They offer business lunch for $100+ per dish plus service charge. If you are looking for a bugget Chinese style meal, this is an good option.'}
{'Restaurant': '我的小館', 'Summary': 'They raised their prices some time ago, which is a pity as their prices was one good thing about it. The boss here is very good, makes foreigners comfortable and they have English menu too! Must try the vegetarian noodle with peanut flavor, its too good. No beef no pork folks, go for egg fried rice o